In [1]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import learning_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Carregar os dados preparados
x_treino = np.load('x_treino.npy')  
y_treino = np.load('y_treino.npy')  
z_treino = np.load('z_treino.npy')

x_validacao = np.load('x_validacao.npy')  
y_validacao = np.load('y_validacao.npy')  
z_validacao = np.load('z_validacao.npy')

x_teste = np.load('x_teste.npy')  
y_teste = np.load('y_teste.npy')  
z_teste = np.load('z_teste.npy')

print(np.shape(x_treino))
print(np.shape(x_validacao))
print(np.shape(x_teste))

In [3]:
def plotar_matriz_confusao(respostas, predicoes, rotulos):
    cm = confusion_matrix(respostas, predicoes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=rotulos, yticklabels=rotulos)
    plt.xlabel('Rótulos previstos')
    plt.ylabel('Rótulos verdadeiros')
    plt.title('Matriz de confusão')
    plt.show()

In [ ]:
# Definir o PCA para reter 95% da variância dos dados
pca = PCA(n_components=0.95)  
x_treino_pca = pca.fit_transform(x_treino)
x_validacao_pca = pca.transform(x_validacao)
x_teste_pca = pca.transform(x_teste)

print("TREINAMENTO")
print(f"Dimensões originais: {np.shape(x_treino)}")
print(f"Dimensões após PCA: {np.shape(x_treino_pca)}")
print("VALIDAÇÃO")
print(f"Dimensões originais: {np.shape(x_validacao)}")
print(f"Dimensões após PCA: {np.shape(x_validacao_pca)}")
print("TESTE")
print(f"Dimensões originais: {np.shape(x_teste)}")
print(f"Dimensões após PCA: {np.shape(x_teste_pca)}")

# KNN com os dados padrão
- n_neighbors: 5
- weights: 'uniform'
- algorithm: 'auto'
- leaf_size: 30
- p: 2
- metric: 'minkowski'
- metric_params: None
- n_jobs: None

In [ ]:
# Código para gerar cruvas de aprendizado.
knn_padrao = KNeighborsClassifier()

train_sizes, train_scores, test_scores = learning_curve(
    knn_padrao, x_treino_pca, y_treino, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10)
)

train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Acurácia no Treinamento', marker='o', color='blue')
plt.plot(train_sizes, test_mean, label='Acurácia no Teste', marker='o', color='orange')
plt.xlabel('Tamanho do Conjunto de Treinamento')
plt.ylabel('Acurácia')
plt.title('Curvas de Aprendizado')
plt.legend()
plt.grid()
plt.show()

In [ ]:
knn_padrao.fit(x_treino_pca, y_treino)

In [ ]:
predicao_validacao_padrao = knn_padrao.predict(x_validacao_pca)
treino_acuracia_padrao = accuracy_score(y_validacao, predicao_validacao_padrao)

print("Acurácia:", treino_acuracia_padrao)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_validacao, predicao_validacao_padrao))

In [ ]:
predicao_teste_padrao = knn_padrao.predict(x_teste_pca)
teste_acuracia_padrao = accuracy_score(y_teste, predicao_teste_padrao)  

print("Acurácia:", teste_acuracia_padrao)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_teste, predicao_teste_padrao))

In [ ]:
# Código para gerar a matriz de confusão.
letras = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]
y_prob_teste = knn_padrao.predict_proba(x_teste_pca)

plotar_matriz_confusao(y_teste, predicao_teste_padrao, letras)


In [ ]:
x_teste_ruidosa = x_teste_pca + np.random.normal(0, 0.1, x_teste_pca.shape)
x_teste_ruidosa = np.clip(x_teste_ruidosa, 0, 1)

y_pred_ruidosa = knn_padrao.predict(x_teste_ruidosa)
teste_acuracia_padrao = accuracy_score(y_teste, y_pred_ruidosa)

print("Acurácia:", teste_acuracia_padrao)

# KNN com GridSearchCV
- n_neighbors: 3
- metric: 'euclidean'

In [ ]:
knn_grid = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [3, 5, 9, 15],
    'metric': ['euclidean', 'manhattan', 'minkowski']  
}

grid_search = GridSearchCV(estimator=knn_grid, param_grid=param_grid, cv=2, scoring='accuracy', verbose=1)
grid_search.fit(x_treino_pca, y_treino)


In [ ]:
print("Melhores Parâmetros:", grid_search.best_params_)
print("Melhor Acurácia na Validação Cruzada:", grid_search.best_score_)

In [ ]:
knn_otimizado = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'], metric=grid_search.best_params_['metric'])

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    knn_otimizado, x_treino_pca, y_treino, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10)
)

train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Acurácia no Treinamento', marker='o', color='blue')
plt.plot(train_sizes, test_mean, label='Acurácia no Teste', marker='o', color='orange')
plt.xlabel('Tamanho do Conjunto de Treinamento')
plt.ylabel('Acurácia')
plt.title('Curvas de Aprendizado')
plt.legend()
plt.grid()
plt.show()

In [ ]:
knn_otimizado.fit(x_treino_pca, y_treino)

In [ ]:
predicao_validacao_otimizado = knn_otimizado.predict(x_validacao_pca)
validacao_acuracia_otimizado = accuracy_score(y_validacao, predicao_validacao_otimizado)
print("Acurácia no Treinamento:", validacao_acuracia_otimizado)
print("Acurácia:", validacao_acuracia_otimizado)

In [ ]:
predicao_teste_otimizado = knn_otimizado.predict(x_teste_pca)
teste_acuracia_otimizado = accuracy_score(y_teste, predicao_teste_otimizado)
print("Acurácia:", teste_acuracia_otimizado)

In [ ]:
# Código para gerar a matriz de confusão.
letras = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]
y_prob_teste = knn_otimizado.predict_proba(x_teste_pca)

plotar_matriz_confusao(y_teste, predicao_teste_otimizado, letras)


In [ ]:
x_teste_ruidosa = x_teste_pca + np.random.normal(0, 0.1, x_teste_pca.shape)
x_teste_ruidosa = np.clip(x_teste_ruidosa, 0, 1)

y_pred_ruidosa = knn_otimizado.predict(x_teste_ruidosa)
teste_acuracia_otimizado = accuracy_score(y_teste, y_pred_ruidosa)

print("Acurácia:", teste_acuracia_otimizado)